<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%9A%D0%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Конечные автоматы Мили и Мура
Вариант 5

In [16]:
import numpy as np
import pandas as pd
import random
import copy
from prettytable import PrettyTable
import itertools as it

In [2]:
def equiv(a,b): # ~
  rez = 0
  if((a == 1 and b == 1) or (a == 0 and b == 0)): rez = 1
  return rez

def implic(a,b): # ->
  rez = 1
  if(a == 1 and b == 0): rez = 0
  return rez

def pier(a,b): # ^
  rez = 0
  if(a == 0 and b == 0): rez = 1
  return rez

def schaeffer(a,b): # |
  rez = 1
  if(a == 1 and b == 1): rez = 0
  return rez

##Построение таблицы истинности:

In [3]:
#таблица истинности
def truthTable(p_inp, p_st, x, v):
  inner_table = [[(str(x1(p_st[i], p_inp[j])) + str(x2(p_st[i], p_inp[j]))) for j in range(len(p_inp))] for i in range(len(p_st))]

  print('Таблица истинности')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "v \ x")
  table.field_names = columns

  for i in range(len(p_inp)):
    table.add_row([p_inp[i], inner_table[i][0], inner_table[i][1],
                             inner_table[i][2], inner_table[i][3]])

  columns.clear()
  print(table)
  return inner_table

#----------------------------------------------------------------------------------------------------

#таблица выходов для автомата Мили
def outputTable_Mili(p_inp, p_st, x, v):
  inner_table = [[(str(y1(p_st[i], p_inp[j])) + str(y2(p_st[i], p_inp[j]))
                  + str(y3(p_st[i], p_inp[j])) + str(y4(p_st[i], p_inp[j]))) for j in range(len(p_inp))] for i in range(len(p_st))]
  #inner_table = [[str(y1(p_st[j], p_inp[0])) for j in range(len(p_inp))],
  #               [str(y2(p_st[j], p_inp[1])) for j in range(len(p_inp))],
  #               [str(y3(p_st[j], p_inp[2])) for j in range(len(p_inp))],
  #               [str(y4(p_st[j], p_inp[3])) for j in range(len(p_inp))]]

  print('Таблица выходов')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "y \ x")
  table.field_names = columns

  for i in range(len(p_inp)):
    table.add_row(['y' + str(i + 1), inner_table[i][0], inner_table[i][1],
                             inner_table[i][2], inner_table[i][3]])

  columns.clear()
  print(table)
  return inner_table

#таблица выходов для автомата Мура
def outputTable_Mura(p_inp, p_st, x, v):
  inner_table = [[(str(y1(p_st[i], p_inp[j])) + str(y2(p_st[i], p_inp[j]))
                  + str(y3(p_st[i], p_inp[j])) + str(y4(p_st[i], p_inp[j]))) for j in range(len(p_inp))] for i in range(len(p_st))]

  print('Таблица выходов')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "y \ x")
  table.field_names = columns

  for i in range(len(p_inp)):
    table.add_row(['y' + str(i + 1), inner_table[i][0], inner_table[i][1],
                             inner_table[i][2], inner_table[i][3]])

  columns.clear()
  print(table)
  return inner_table

  #таблица выходов
def outputTable(p_inp, p_st, x, v):
  inner_table = [[(str(y1(p_st[i], p_inp[j])) + str(y2(p_st[i], p_inp[j]))
                  + str(y3(p_st[i], p_inp[j])) + str(y4(p_st[i], p_inp[j]))) for j in range(len(p_inp))] for i in range(len(p_st))]

  print('Таблица выходов')
  table = PrettyTable()
  columns = p_st.copy()
  columns.insert(0, "v \ x")
  table.field_names = columns

  for i in range(len(p_inp)):
    table.add_row([p_inp[i], inner_table[i][0], inner_table[i][1],
                             inner_table[i][2], inner_table[i][3]])

  columns.clear()
  print(table)
  return inner_table

#----------------------------------------------------------------------------------------------------

#матрица соединений
def creatly_connection_table(trans, out):
    table = pd.DataFrame('0', index=range(trans.shape[1]-1), columns=trans.columns)

    for i in range(table.shape[0]):
        table['input'][i] = table.columns[i]

    for i in table.columns[0:table.columns.shape[0]-1]:
        for j in range(table.shape[0]):
            cell = trans.index[trans[i] == table[table.columns[table.shape[1]-1] ][j]]
            if len(cell) != 0:
                table[i][j] = '';
                for k in range(len(cell)):
                    table[i][j] += out[out.columns[out.shape[1]-1]][cell[k]] + '/' + out[i][cell[k]]
                    if k != len(cell)-1:
                        table[i][j] += ' v '

    return table

###Задача 1. Мили

In [4]:
x0 = [0, 1]

#2 входа - v
v = [0, 1]
#2 состояния - x1/x2
x = [0, 1]
#4 выхода - y1/y2/y3/y4

#+(сумма по модулю 22), ^(стрелка пирса), & и v(конъюнкция и дизъюнкция)
#|(штрих шеффера), '(отрицание)
#->(импликация) и ~(эквивалентность)

In [5]:
pos_inputs = []
for i in range(len(v)):
  for j in range(len(v)):
    pos_inputs.append(str(v[i]) + str(v[j]))
print('Возможные входы:', pos_inputs)

pos_states = []
for i in range(len(x)):
  for j in range(len(x)):
    pos_states.append(str(x[i]) + str(x[j]))
print('Возможные состояния:', pos_states)

Возможные входы: ['00', '01', '10', '11']
Возможные состояния: ['00', '01', '10', '11']


In [6]:
def x1(x, v):
  #v1 + (x2 | x1 + v2 v (v2 + (x2 ^ x2)))
  rezult = int(v[0]) ^ (schaeffer(int(x[1]), int(x[0])) ^ int(v[1]) or (int(v[1]) ^ pier(int(x[1]), int(x[1]))))
  return rezult

def x2(x, v):
  #v1 -> v1 | (x1 ' | v1 + x1 ^ v2)
  rezult = schaeffer(implic(int(v[0]), int(v[0])), pier(schaeffer(int(x[0]), not int(v[0])) ^ int(x[0]), int(v[1])))
  return rezult

def y1(x, v):
  #x1 v x2 + x2 | x1 ~ x1 ~ (x2 ' -> x1)
  rezult = equiv(equiv(schaeffer(int(x[0]) or int(x[1]) ^ int(x[1]), int(x[0])), int(x[0])), implic(int(x[1]), not int(x[0])))
  return rezult

def y2(x, v):
  #x1 ~ x2 ^ (x1 + x2 | (x2 ~ x2))
  rezult = pier(equiv(int(x[0]), int(x[1])), schaeffer(int(x[0]) ^ int(x[1]), equiv(int(x[1]), int(x[1]))))
  return rezult

def y3(x, v):
  #x2 & x2 | x2 + x2 ^ x2
  rezult = pier(schaeffer(int(x[1]) and int(x[1]), int(x[1])) ^ int(x[1]), int(x[1]))
  return rezult

def y4(x, v):
  #x1 v x1 ~ x2 & (x2 ^ x1 ^ x2)
  rezult = equiv(int(x[0]) or int(x[0]), int(x[1]) and pier(pier(int(x[1]), int(x[0])), int(x[1])))
  return rezult

In [7]:
truth_table = truthTable(pos_inputs, pos_states, x, v)

Таблица истинности
+-------+----+----+----+----+
| v \ x | 00 | 01 | 10 | 11 |
+-------+----+----+----+----+
|   00  | 11 | 01 | 01 | 11 |
|   01  | 11 | 11 | 01 | 01 |
|   10  | 11 | 01 | 00 | 11 |
|   11  | 01 | 11 | 10 | 01 |
+-------+----+----+----+----+


In [8]:
#переделать таблицу(в строках y1,y2,y3,y4)
#output_table = outputTable_Mili(pos_inputs, pos_states, x, v)
output_table = outputTable(pos_inputs, pos_states, x, v)

Таблица выходов
+-------+------+------+------+------+
| v \ x |  00  |  01  |  10  |  11  |
+-------+------+------+------+------+
|   00  | 0001 | 0001 | 0001 | 0001 |
|   01  | 0101 | 0101 | 0101 | 0101 |
|   10  | 0100 | 0100 | 0100 | 0100 |
|   11  | 1000 | 1000 | 1000 | 1000 |
+-------+------+------+------+------+


In [9]:
#Для удобства и простоты построения матрицы соединений, можно использовать тип DF
truth_copy = pd.DataFrame(truth_table.copy(), columns = pos_states)
truth_copy['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=truth_copy.index)

out_copy = pd.DataFrame(output_table.copy(), columns = pos_states)
out_copy['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=out_copy.index)

creatly_connection_table(truth_copy, out_copy)

,00,01,10,11,input
0,0,0,10/0100,0,00
1,11/1000,00/0001 v 10/0100,00/0001 v 01/0101,01/0101 v 11/1000,01
2,0,0,11/1000,0,10
3,00/0001 v 01/0101 v 10/0100,01/0101 v 11/1000,0,00/0001 v 10/0100,11


###Задача 2. Мура

In [10]:
x0_2 = [0, 0]
#2 входа - v
#v = [0, 1]
#2 состояния - x
#4 выхода - y

In [11]:
def x1(x, v):
  #v1 -> (v2 & (x2 v v1 v v1 ^ x2 + (v2 -> x1 ~v2)))
  rezult = implic(int(v[0]), int(v[1]) and (pier(int(x[1]) or int(v[0]) or int(v[0]), int(x[1])) ^ equiv(implic(int(v[1]), int(x[0])), int(v[1]))))
  return rezult

def x2(x, v):
  #x1 | (x2 ^ v1 ' | (v1 ~ x1 + (x1 v v2)))
  rezult = schaeffer(int(x[0]), schaeffer(pier(int(x[1]), not int(v[0])), (equiv(int(v[0]), int(x[0])) ^ (int(x[0]) or int(v[1])))))
  return rezult

def y1(x, v):
  #x1 + v1 -> (x2 ^ v1 | (x1'))
  rezult = int(x[0]) ^ implic(v[0], pier(x[1], schaeffer(v[0], not x[0])))
  return rezult

def y2(x, v):
  #x2 v v2' -> (x2~v1)
  rezult = implic(x[1] and (not v[1]), equiv(x[1], v[0]))
  return rezult

def y3(x, v):
  #x1 & (x1 v (x2 & (x2 ^ (x1 & x2 + x1 | v2 & x1))))
  rezult = x[0] and (x[0] or (x[1] and pier(x[1], schaeffer(x[0] and x[1] ^ x[0], v[1] and x[0]))))
  return rezult

def y4(x, v):
  #v1 ^ x1' v (x1 v x2 -> x2 & (x2'))
  rezult = pier(v[0], not x[0]) or (implic(x[0] or x[1], x[1] and (not x[1])))
  return rezult

In [12]:
truth_table2 = truthTable(pos_inputs, pos_states, x, v)

Таблица истинности
+-------+----+----+----+----+
| v \ x | 00 | 01 | 10 | 11 |
+-------+----+----+----+----+
|   00  | 11 | 11 | 01 | 01 |
|   01  | 11 | 11 | 01 | 01 |
|   10  | 10 | 10 | 00 | 10 |
|   11  | 10 | 10 | 00 | 10 |
+-------+----+----+----+----+


In [13]:
output_table2 = outputTable(pos_inputs, pos_states, x, v)

Таблица выходов
+-------+------+------+------+------+
| v \ x |  00  |  01  |  10  |  11  |
+-------+------+------+------+------+
|   00  | 1101 | 1101 | 1101 | 1101 |
|   01  | 1101 | 1101 | 1101 | 1101 |
|   10  | 0111 | 0111 | 0111 | 0111 |
|   11  | 0111 | 0111 | 0111 | 0111 |
+-------+------+------+------+------+


In [14]:
#Для удобства и простоты построения матрицы соединений, можно использовать тип DF
truth_copy2 = pd.DataFrame(truth_table2.copy(), columns = pos_states)
truth_copy2['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=truth_copy2.index)

out_copy2 = pd.DataFrame(output_table2.copy(), columns = pos_states)
out_copy2['input'] = pd.Series([''.join(pos_inputs[i]) for i in range(len(pos_inputs))], index=out_copy2.index)

creatly_connection_table(truth_copy2, out_copy2)

,00,01,10,11,input
0,0,0,10/0111 v 11/0111,0,00
1,0,0,00/1101 v 01/1101,00/1101 v 01/1101,01
2,10/0111 v 11/0111,10/0111 v 11/0111,0,10/0111 v 11/0111,10
3,00/1101 v 01/1101,00/1101 v 01/1101,0,0,11


##СКНФ/СДНФ


In [17]:
table_combinations = pd.DataFrame(it.product([0,1], repeat=3))
table_combinations

,0,1,2
0,0,0,0
1,0,0,1
2,0,1,0
3,0,1,1
4,1,0,0
5,1,0,1
6,1,1,0
7,1,1,1


###Задача 3.

In [15]:
x0_3 = [1, 1]

phi = [['11', '01', '10', '10'], ['10', '00', '10', '10'], ['11', '01', '11', '11'], ['11', '01', '10', '10']]
psi = [['1101', '1110', '1100', '1110'], ['1010', '0111', '1111', '0111'], ['1001', '1110', '1101', '1111'], ['1000', '1101', '1101', '1101']]

###Задача 4.

In [ ]:
x0_4 = [0, 1]

phi_2 = [['00', '00', '10', '10'], ['10', '10', '10', '10'], ['01', '01', '11', '11'], ['00', '00', '10', '10']]
psi_2 = ['011', '010', '101', '011']